In [1]:
import os
import glob
import pandas as pd
import numpy as np
import folium

### Import streamflow data, quality flags and metadata

In [2]:
streamflow_data = "//gmvstorage.gmv.es/storage/anin/Streamflow/processed_data/streamflow_dataset.xlsx"
df_streamflow = pd.read_excel(streamflow_data, index_col=0)
quality_data ="//gmvstorage.gmv.es/storage/anin/Streamflow/processed_data/Streamflow_quality.xlsx" 
df_quality = pd.read_excel(quality_data, index_col=0)

metadata = "//gmvstorage.gmv.es/storage/anin/Streamflow/All stations_v2.xlsx"
df_metadata = pd.read_excel(metadata)

## Station selection based on quality flags

#### Quality categories extracted from the excel file "Quality codes"

In [3]:
usable =[1,2,3,7,27,47,50,60,64,80,130,150,250]
unsure = [5,6,26,59,65,66,84,91,92,140,152,153,154,155,163,164,165,171,172,173,246,247,248,249,251,252,253,254]
unusable = [4,70,78,79,100,151,160,161,162,170,201,255]

####  Reclassify quality data

In [4]:
# Function to classify values
def classify_value(value):
    if value in usable:
        return 'Usable'
    elif value in unsure:
        return 'Unsure'
    elif value in unusable:
        return 'Unusable'
    else:
        return 'Unknown'

# Create a new dataframe with classified values
classified_df = df_quality.applymap(classify_value)

In [5]:
# Find columns with 80% or more "usable" values
usable_columns = classified_df.columns[(classified_df == 'Usable').mean() >= 0.75]

# Create a new dataframe with the selected columns
usable_df = classified_df[usable_columns].copy()

# Set non-"usable" values to NaN
usable_df[usable_df != 'usable'] = np.nan

print("Number of selected stations:",len(usable_columns))

Number of selected stations: 335


### Extract metadata of selected stations

In [6]:
# Select metadatadata from selected stations in "usable_columns"

station_list = list(usable_columns)
usable_metadata = df_metadata[df_metadata['STATION'].isin(station_list)]
print(usable_metadata)

# Export the cleaned streamflow DataFrame to an Excel file
out_path = "//gmvstorage.gmv.es/storage/anin/Streamflow/processed_data/"
fNout = os.path.join(out_path, "clean_metadata.xlsx")
usable_metadata.to_excel(fNout, index=True)



    STATION                         STNAME             SHORTNAME   LATITUDE  \
0    A2H006     Pienaars River @ Klipdrift  Pienaars @ Klipdrift -25.380220   
1    A2H012    Krokodil River @ Kalkheuwel  Krokodil @ Kalkheuwe -25.810560   
2    A2H013   Magalies River @ Scheerpoort  Magalies @ Scheerpoo -25.777030   
3    A2H014     Hennops River @ Skurweberg  Hennops @ Skurweberg -25.798280   
4    A2H021  Pienaars River @ Buffelspoort  Pienaars @ Buffelspo -25.128560   
..      ...                            ...                   ...        ...   
543  X3H002      Klein Sabie River @ Sabie   Klein Sabie @ Sabie -25.088000   
544  X3H003   Mac-Mac River @ Geelhoutboom  Mac-Mac @ Geelhoutbo -24.993278   
545  X3H004      Noordsand River @ De Rust   Noordsand @ De Rust -25.076194   
546  X3H008            Sand River @ Exeter         Sand @ Exeter -24.770028   
550  X4H004  Nwanedzi River @ Wenela Drift     Nwanedzi @ Wenela -24.449722   

     LONGITUDE   COMMENCE  REGION Province  WMA  
0

### Create interactive map of stations locations

In [7]:
 # Create a map centered at the mean coordinates of the data
 center_lat = df_metadata["LATITUDE"].mean()
 center_lon = df_metadata["LONGITUDE"].mean()
 m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

 # Add markers for each data point
 for index, row in df_metadata.iterrows():
     lat = row["LATITUDE"]
     lon = row["LONGITUDE"]
     folium.Marker(location=[lat, lon]).add_to(m)
 m

### Select stations in the streamflow database based on the resuults from the quality control

In [13]:
# Select Stations
usable_streamflow = df_streamflow[station_list]

# Set values non regarded as "usable" to NaN (Align NaN values in usable_streamflow with usable_df)
usable_streamflow = usable_streamflow.fillna(usable_df)
usable_streamflow

,C5H007,C5H012,C5H014,C5H022,D1H001,D1H003,D1H006,D1H009,D1H011,D2H012,...,B7H013,B7H014,B8H008,B8H009,B8H010,B8H014,B8H017,B8H019,B9H001,B9H002
1979-10-31,0.0,1.72,NaN,NaN,0.047,251.219,38.908,230.978,34.943,1.567,...,NaN,0.082,2.443,1.632,0.49,0.809,0.674,NaN,NaN,NaN
1979-11-30,0.0,0.0,NaN,NaN,NaN,86.176,NaN,NaN,8.127,2.562,...,0.833,0.054,NaN,1.184,0.752,0.724,0.37,NaN,NaN,NaN
1979-12-31,0.0,0.0,NaN,NaN,NaN,159.114,NaN,141.162,15.446,2.495,...,0.953,NaN,5.084,2.932,1.875,1.442,NaN,NaN,NaN,NaN
1980-01-31,0.0,0.0,NaN,NaN,0.006,109.236,5.644,NaN,NaN,0.723,...,NaN,NaN,7.083,3.183,3.514,2.248,9.055,NaN,NaN,NaN
1980-02-29,0.059,0.006,NaN,NaN,0.301,144.546,9.395,NaN,10.461,1.176,...,2.602,0.309,8.583,7.981,11.611,5.415,38.803,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Compare column names to ensure that station selection is correct
columns_equal = usable_df.columns.equals(usable_streamflow.columns)

print('Quality control performed correctly =',columns_equal)

Quality control performed correctly = True


In [16]:
# Export the cleaned streamflow DataFrame to an Excel file
out_path = "//gmvstorage.gmv.es/storage/anin/Streamflow/processed_data/"
fNout = os.path.join(out_path, "clean_streamflow.xlsx")
usable_streamflow.to_excel(fNout, index=True)

In [ ]:
# Plot the time series to ensure that there are no huge artifacts
import matplotlib.pyplot as plt

# Create a subdirectory for the plots if it doesn't exist
plot_dir = "//gmvstorage.gmv.es/storage/anin/Streamflow/processed_data/plots/"
os.makedirs(plot_dir, exist_ok=True)

# Generate and save a plot for each column
for column in usable_streamflow.columns:
    fig, ax = plt.subplots()
    ax.plot(usable_streamflow.index, usable_streamflow[column])
    ax.set_xlabel('Date')
    ax.set_ylabel(column)
    ax.set_title(column)
    plt.savefig(os.path.join(plot_dir, f"{column}.png"))
    plt.close()